In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow.keras import layers,datasets
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
from tensorflow import keras
import gc
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
import category_encoders as ce
from sklearn.metrics import f1_score,classification_report,roc_auc_score
from impala.dbapi import connect
from impala.util import as_pandas
import pymysql
import ast
import requests
import random

In [82]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [83]:
def get_game_id():
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1136 AND dev_game_id IS NOT NULL 
    '''
    cur.execute(sql)
    result_df = cur.fetchall()
    cur.close()
    conn.close()
    return result_df

In [84]:
# 获取近期所有计划
def get_plan_info():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = pymysql.connect(host='db-slave-modeltoufang-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
        SELECT
                * 
            FROM
                db_ptom.ptom_third_plan p
            WHERE
                game_id IN ({})
                AND media_id = 45
                AND opt_status = 1
                AND create_time>=date( NOW() - INTERVAL 1440 HOUR )
                AND create_time<= date(NOW())
                            AND plan_id >= (
                                select plan_id from db_ptom.ptom_plan
                                where create_time >= date( NOW() - INTERVAL 1440 HOUR )
                                and create_time <= date( NOW() - INTERVAL 1416 HOUR )
                                limit 1
                            )
    '''
    finalSql = sql.format(game_id)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    return result_df

In [85]:
# 获取image_id,label_ids
def get_image_info():
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id,
            a.image_id,
            b.label_ids
        FROM
            db_data_ptom.ptom_plan a
            LEFT JOIN
            db_data_ptom.ptom_image_info b
        on a.image_id = b.image_id
        WHERE
            a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1136 AND dev_game_id IS NOT NULL ) 
            AND a.create_time >= date( NOW() - INTERVAL 2880 HOUR ) 
        GROUP BY
            a.chl_user_id,
            a.source_id,
            a.image_id
    '''
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)
    result_df = result_df.dropna(axis=0, how='any')
    result_df['channel_id'] = result_df['channel_id'].map(int)
    result_df['source_id'] = result_df['source_id'].map(int)
    result_df['image_id'] = result_df['image_id'].map(int)
    cur.close()
    conn.close()
    return result_df

In [86]:
def get_launch_report():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id AS source_id,
            b.tdate,
            b.amount,
            b.new_role_money,
            b.new_role_money / b.amount as roi,
            b.pay_role_user_num / b.create_role_num as pay_rate
        FROM
            db_data_ptom.ptom_plan a
            LEFT JOIN db_stdata.st_lauch_report b ON a.chl_user_id = b.channel_id 
            AND a.source_id = b.source_id 
        WHERE
            a.create_time >= date( NOW() - INTERVAL 1440 HOUR ) 
            AND b.tdate >= date( NOW() - INTERVAL 1440 HOUR ) 
            AND b.tdate_type = 'day' 
            AND b.media_id = 45
            AND b.game_id IN ({})
            AND b.amount >= 500
    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    result_df['tdate'] = pd.to_datetime(result_df['tdate'])
    result_df = result_df.sort_values('tdate')
    result_df = result_df.drop_duplicates(['channel_id', 'source_id'], keep='first')

    return result_df

In [87]:
# 获取近期优化计划的创意数据
def get_creative():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modeltoufang-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/ 
        SELECT
            a.chl_user_id 'channel_id',
            a.source_id,
            a.image_id,
            b.creative_info 
        FROM
            (
            SELECT
                a.chl_user_id,
                a.source_id,
                a.image_id,
                b.ad_id,
                b.ad_account_id 
            FROM
                db_ptom.ptom_plan a
                LEFT JOIN db_ptom.ptom_third_plan b ON a.plan_id = b.plan_id 
            WHERE
                a.media_id = 45 
                AND a.create_time >= date( NOW() - INTERVAL 1440 HOUR ) 
                AND a.game_id IN ({}) 
                AND b.ad_id IS NOT NULL 
            ) a
            INNER JOIN db_ptom.ptom_third_ad_creative b ON a.ad_id = b.ad_id 
            AND a.ad_account_id = b.ad_account_id 
        WHERE
            b.create_time >= date( NOW() - INTERVAL 1440 HOUR ) 
            AND b.media_id = 45 
            AND a.image_id IS NOT NULL
            

    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    return result

In [88]:
def get_plan_json(plan_info):
    plan_info['ad_info'] = plan_info['ad_info'].apply(json.loads)
    temp = plan_info['ad_info'].apply(pd.Series)
    plan_info = pd.concat([plan_info,temp], axis=1)
    plan_info.drop('ad_info', axis=1, inplace=True)

    temp = plan_info['audience'].apply(pd.Series)
    plan_info = pd.concat([plan_info,temp], axis=1)
    plan_info.drop('audience', axis=1, inplace=True)

    temp = plan_info['ocpc'].apply(pd.Series)
    plan_info = pd.concat([plan_info,temp], axis=1)
    plan_info.drop('ocpc', axis=1, inplace=True)

    plan_info = plan_info[['ad_account_id', 'game_id', 'channel_id', 'source_id', 'create_time',
           'bidtype', 'producttypes', 'ftypes', 'pause', 'autoExpansion',
           'education', 'customAge', 'keywords', 'keywordsExtend', 'newInterests',
           'sex', 'iosVersion', 'androidVersion', 'excludeTrans', 'region',
           'device', 'age', 'transType', 'ocpcLevel', 'isSkipStageOne', 'payMode',
           'optimizeDeepTrans', 'transFrom', 'ocpcBid', 'deepOcpcBid',
           'deepTransType', 'roiRatio', 'useRoi']]
    
    # plan_info 去重
    plan_info = plan_info.sort_values(by='create_time')
    plan_info = plan_info.drop_duplicates(subset=['channel_id','source_id'], keep='last')
    return plan_info

In [89]:
def get_train_df():
    plan_info = get_plan_info()
    plan_info = get_plan_json(plan_info)
    creative_info = get_creative()
    creative_info = creative_info.drop_duplicates(subset=['channel_id','source_id'], keep='last')
    creative_info['creative_info'] = creative_info['creative_info'].apply(json.loads)
    temp = creative_info['creative_info'].apply(pd.Series)
    if 'image_id' in temp.columns:
        temp = temp.drop('image_id', axis=1)
    temp = temp[['materialstyle','material']]
    temp['material'] = temp['material'].apply(json.loads)
    creative_info = pd.concat([creative_info,temp], axis=1)
    creative_info.drop('creative_info', axis=1, inplace=True)
    temp = creative_info['material'].apply(pd.Series)
    temp = temp[['brand','userPortrait','elements','title','subtitle']]
    creative_info = pd.concat([creative_info,temp], axis=1)
    creative_info.drop('material', axis=1, inplace=True)
    creative_info_1 = creative_info[creative_info['elements'].isna()]
    creative_info_2 = creative_info[creative_info['elements'].notna()]
    if creative_info_1.shape[0] != 0:
        creative_info_1['titles'] = creative_info_1['title'].apply(lambda x: [x])
    creative_info_2['titles'] = creative_info_2['elements'].apply(lambda x: x['titles'])
    creative_info = pd.concat([creative_info_1,creative_info_2],axis=0)
    creative_info.drop(['elements','title'],axis=1,inplace=True)
    creative_info['subtitle'] = creative_info['subtitle'].fillna(creative_info['subtitle'].mode()[0])

    plan_info = pd.merge(plan_info, creative_info, on=['channel_id','source_id'], how='inner')
    image_info = get_image_info()
    launch_report = get_launch_report()
    
    plan_info['create_time'] = pd.to_datetime(plan_info['create_time'])
    plan_info_current = plan_info[plan_info['create_time']>=pd.datetime.now() - pd.DateOffset(30)]
    image_info = image_info[image_info['image_id'].notna()]
    image_info['image_id'] = image_info['image_id'].astype(int)
    
    score_image_1 = get_score_image_other()
    score_image_2 = get_score_image()
    score_image = np.union1d(score_image_1,score_image_2)
    score_image = filter_game_image(score_image, 1001932)
    
    now_plan_roi = get_now_plan_roi()
    df_create = pd.merge(plan_info_current,image_info,on=['channel_id','source_id','image_id'],how='left')
    df_create = pd.merge(df_create,now_plan_roi.drop(['ad_account_id'], axis=1),on=['channel_id','source_id'],how='inner')
    # df_create['platform'] = df_create['platform'].astype(str)
    # df_create['platform'] = df_create['platform'].map({"['ANDROID']":1,"['IOS']":2})
    # df_create['platform'] = df_create['platform'].astype(int)
    df_create.to_csv('./df_create.csv',index=0)
    # 只跑付费ROI，不跑激活付费
#     df_create = df_create[df_create['optimizeDeepTrans']==False] 
    plan_create = create_plan(df_create,score_image)
    
    return plan_create

In [90]:
# 获取近期计划的运营数据
def get_now_plan_roi():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
        SELECT
            a.ad_account_id,
            b.channel_id,
            b.source_id,
            b.tdate,
            b.amount,
            b.new_role_money,
            b.new_role_money / b.amount AS roi,
            b.pay_role_user_num / b.create_role_num AS pay_rate 
        FROM
            db_data_ptom.ptom_plan a 
        left join
            db_stdata.st_lauch_report b
        on a.chl_user_id=b.channel_id and a.source_id=b.source_id
        WHERE
            b.tdate >= date( NOW() - INTERVAL 240 HOUR ) 
            AND b.tdate_type = 'day' 
            AND b.media_id = 45 
            AND b.game_id IN ({}) 
            AND b.amount >= 200 
            AND b.platform = 1
            AND b.pay_role_user_num >= 1 
            AND b.new_role_money >= 6
            AND (b.new_role_money / b.amount)>=0.005
    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    result_df['tdate'] = pd.to_datetime(result_df['tdate'])
    result_df = result_df.sort_values('tdate')
    result_df = result_df.drop_duplicates(['channel_id', 'source_id'], keep='first')
#     result_df = result_df[result_df['roi'] >= 0.03]
    return result_df

In [91]:
# 获取score_image (分数大于550的image_id)
def get_score_image():
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                    password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql_queue = 'set tez.queue.name=offline'
    sql = '''
            SELECT
            image_id,
            label_ids 
        FROM
            dws.dws_image_score_d 
        WHERE
            dt = CURRENT_DATE 
            AND media_id = 45 
            AND image_create_role_roi >= 0.01 
            AND ( image_create_role_retain_1d >= 0.1 OR image_create_role_retain_1d IS NULL OR image_create_role_retain_1d=0) 
            AND score >= 500 
        GROUP BY
            image_id,
            label_ids
    '''
    
    cursor.execute(sql_engine)
    cursor.execute(sql_queue)
    cursor.execute(sql)
    result = as_pandas(cursor)
    result['label_ids'] = result['label_ids'].astype(str)
    result['label_ids'] = result['label_ids'].apply(lambda x: x.strip('-1;') if '-1' in x else x)
    result['label_ids'] = pd.to_numeric(result['label_ids'],errors='coerce')
#     result = result[result['label_ids'].isin([27])]
#     print(result)
    # 关闭链接
    cursor.close()
    conn.close()
    
    return result['image_id'].values

In [92]:
# 560  # 0.012
def get_score_image_other():
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                    password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql_queue = 'set tez.queue.name=offline'
    sql = '''
        SELECT
            image_id,
            label_ids 
        FROM
            dws.dws_image_score_d 
        WHERE
            dt = CURRENT_DATE 
            AND media_id in (10,16,32) 
            AND image_create_role_roi >= 0.012   
            AND ( image_create_role_retain_1d >= 0.1 OR image_create_role_retain_1d IS NULL OR image_create_role_retain_1d=0) 
            AND score >= 550      
        GROUP BY
            image_id,
            label_ids
    '''
    cursor.execute(sql_engine)
    cursor.execute(sql_queue)
    cursor.execute(sql)
    result = as_pandas(cursor)
    result['label_ids'] = result['label_ids'].astype(str)
    result['label_ids'] = result['label_ids'].apply(lambda x: x.strip('-1;') if '-1' in x else x)
    result['label_ids'] = pd.to_numeric(result['label_ids'],errors='coerce')
#     result = result[result['label_ids'].isin([27])]
#     print(result)
    # 关闭链接
    cursor.close()
    conn.close()
    
    return result['image_id'].values

In [93]:
def filter_exclude_image(image_ids, media_id):
    image_ids = [str(i) for i in image_ids]
    image_ids = ','.join(image_ids)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            image_id,
            image_name
        FROM
            db_data_ptom.ptom_image_info
        WHERE
            image_id IN ({})
            AND NOT FIND_IN_SET({}, exclude_media_ids)
    '''
    result = pd.read_sql(sql.format(image_ids, media_id), conn)
    # result = result[result['image_name'].apply(lambda x:True if len(re.findall(r"QMZZ(.*?)-", x))>0 and int(re.findall(r"QMZZ(.*?)-", x)[0]) not in tt_ban else False)]
    # 关闭链接
    cur.close()
    conn.close()

    return result['image_id'].values

In [94]:
def filter_game_image(image_ids, game_id):
    image_ids = [str(i) for i in image_ids]
    image_ids = ','.join(image_ids)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            image_id,
            image_name
        FROM
            db_data_ptom.ptom_image_info
        WHERE
            image_id IN ({})
            AND FIND_IN_SET({}, game_ids)
    '''
    cur.execute(sql.format(image_ids, game_id))
    result = pd.read_sql(sql.format(image_ids, game_id), conn)
    # result = result[result['image_name'].apply(lambda x:True if len(re.findall(r"QMZZ(.*?)-", x))>0 and int(re.findall(r"QMZZ(.*?)-", x)[0]) not in tt_ban else False)]
    # 关闭链接
    cur.close()
    conn.close()

    return result['image_id'].values

In [95]:
# 对长期累积roi不达标的素材进行过滤
# 获取数据-1-6天
def get_data_1_6():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                   password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql = '''
            SELECT
                user_id,
                game_id,
                channel_id,
                source_id,
                role_id,
                platform,
                media_id,
                pay_num,
                pay_sum,
                create_role_time,
                created_role_day,
                pay_7_pred,
                dt 
            FROM
                tmp_data.tmp_roles_portrait_info_predict 
            WHERE
                dt = CURRENT_DATE
                AND channel_id = 21206
                AND game_id IN ({})
    
    '''
    finalSql = sql.format(game_id)
    cursor.execute(finalSql)
    result = as_pandas(cursor)

    # 关闭链接
    cursor.close()
    conn.close()

    return result

def get_data_7():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hadoop',
                   password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql = '''
        SELECT
            user_id,
            game_id,
            channel_id,
            source_id,
            role_id,
            platform,
            media_id,
            pay_num,
            pay_sum,
            create_role_time,
            created_role_day,
            pay_sum AS pay_7_pred,
            dt 
        FROM
            tmp_data.tmp_roles_portrait_info_train2 
        WHERE
            dt = CURRENT_DATE 
            AND created_role_day = 7
            AND channel_id = 21206
            AND game_id IN ({})
    '''
    finalSql = sql.format(game_id)
    cursor.execute(finalSql)
    result = as_pandas(cursor)

    # 关闭链接
    cursor.close()
    conn.close()

    return result

# 获取近期所有计划的消耗情况
def get_amount_info():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
            SELECT
                tdate,
                channel_id,
                source_id,
                game_id,
                media_id,
                platform,
                amount 
            FROM
                db_stdata.st_lauch_report 
            WHERE
                game_id IN ({}) 
                AND tdate_type = 'day' 
                AND channel_id = 21206
                AND tdate >= date( NOW() - INTERVAL 7 DAY )
                AND tdate <= date( NOW() - INTERVAL 1 DAY )
    '''
    finalSql = sql.format(game_id)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    
    return result_df

def calculate_roi(groupby_list=['media_id','platform','image_id']):
    # 获取预测数据
    df_1 = get_data_7()
    df_2 = get_data_1_6()
    df = df_1.append(df_2)
    df['create_role_time'] = pd.to_datetime(df['create_role_time']).dt.date
    # 聚合生成计划维度的回款预测数据
    source_df = pd.DataFrame({'7_pay_sum':df.groupby(['game_id','channel_id','source_id','create_role_time','media_id','platform'])['pay_7_pred'].sum()}).reset_index()
    source_df['media_id'] = source_df['media_id'].map(int)
    source_df = source_df[source_df['media_id'].isin([10,16,32,45])]
    # 获取计划消耗数据，只取4个主流媒体
    amount_info = get_amount_info()
    amount_info = amount_info[amount_info['media_id'].isin([10,16,32,45])]
    image_info = get_image_info()
    # 拼接数据
    source_df.rename(columns={'create_role_time':'tdate'}, inplace=True)
    source_df['tdate'] = pd.to_datetime(source_df['tdate'])
    source_df = pd.merge(source_df,amount_info,on=['channel_id','source_id','tdate','media_id','platform','game_id'],how='outer')
    source_df['amount'] = source_df['amount'].fillna(0)
    source_df['7_pay_sum'] = source_df['7_pay_sum'].fillna(0)
    data = pd.merge(source_df,image_info,on=['channel_id','source_id'],how='left')
    
    df_pay_sum = pd.DataFrame({'7_pay_sum': data.groupby(groupby_list)['7_pay_sum'].sum()}).reset_index()
    df_amount = pd.DataFrame({'7_amount': data.groupby(groupby_list)['amount'].sum()}).reset_index()
    df = pd.merge(df_amount,df_pay_sum,on=groupby_list,how='left')
    df['7_pay_sum'] = df['7_pay_sum'].apply(lambda x: round(x,2))
    df['7_amount'] = df['7_amount'].apply(lambda x: round(x,2))
    df = df[df['7_amount']>0]
    df['7_roi'] = round(df['7_pay_sum'] / df['7_amount'],4)
    
    df = df[(df['media_id']==45)&(df['platform']==1)&(df['7_amount']>=15000)&(df['7_roi']<0.1)]
    df['image_id'] = df['image_id'].astype(int)
    return df['image_id'].values

In [96]:
calculate_roi()

array([], dtype=int32)

In [97]:
def create_plan(df_create,score_image):
    # 选ad_account_id、image_id每个账号+素材8条
    game_id = 1001932  ## 选择包：
#     df_create = df_create[df_create['game_id'].isin([1001794, 1001862, 1001783, 1001863, 1001961, 1001935])]
#     image_id_group = np.intersect1d(df_create['image_id'].unique(), score_image)
    image_id_group = filter_exclude_image(score_image, 45)
    image_id_group = np.setdiff1d(image_id_group,calculate_roi())

    print('匹配的素材ID', image_id_group)
#     df_create = df_create[df_create['image_id'].isin(image_id_group)]
    
    # 选择预算，不限制预算
    plan = pd.DataFrame({'image_id': image_id_group})
    
    sample_df = df_create[['producttypes', 'ftypes','education', 'customAge', 'keywords', 'keywordsExtend', 'newInterests',
                       'sex', 'iosVersion', 'androidVersion', 'excludeTrans', 'region',
                       'device', 'age','transType', 'ocpcLevel', 'isSkipStageOne', 'payMode',
                   'optimizeDeepTrans', 'transFrom', 'ocpcBid', 'deepOcpcBid',
                   'deepTransType', 'roiRatio', 'useRoi', 'materialstyle',
                   'brand', 'userPortrait','titles', 'subtitle']]
    sample_df = sample_df.sample(n=plan.shape[0], replace=True).reset_index(drop=True)
    plan = pd.concat([plan, sample_df], axis=1)

    plan = plan.reset_index(drop=True)
    plan['game_id'] = game_id
    return plan

In [98]:
def get_plan(plan_num, ad_account_id_group):
    plan_create = get_train_df()  
    # 每个账号分配计划数量
    plan_result = pd.DataFrame()
    for ad_account_id in ad_account_id_group:
        if plan_create.shape[0] < plan_num:
            plan_num = plan_create.shape[0]
        plan_result_ = plan_create.sample(n=plan_num, replace=False).reset_index(drop=True)
    #     plan_create = plan_create.drop(plan_create[plan_create['image_id'].isin(plan_result_.image_id.values)].index, axis=0)
        plan_result_['ad_account_id'] = ad_account_id
        plan_result = plan_result.append(plan_result_)   
    plan_result['autoExpansion'] = 0
    plan_result['pause'] = False
    
    plan_result = plan_result.reset_index(drop=True)
    plan_result['deepTransTypes'] = plan_result.apply(lambda x: [26],axis=1)   # 转化参数
    plan_result['brand'] = plan_result.apply(lambda x: '江南杂货铺' ,axis=1)
    plan_result['userPortrait'] = "https://feed-image.baidu.com/0/pic/-905438858_796527356_-1984841761.jpg"  # 品牌头像
    plan_result['plan_auto_task_id'] = "12550,12594"
    plan_result['op_id'] = 13268
    plan_result['flag'] = 'BD'
    plan_result['game_name'] = '大东家'
#     plan_result['ocpcBid'] = plan_result['ocpcBid'].apply(lambda x: random.randint(4200.00, 4400.00))
#     plan_result['roiRatio'] = 0.05
#     plan_result['useRoi'] = True     # ROI系数

    # 调出价
    plan_result['ocpcBid'] = plan_result['optimizeDeepTrans'].apply(lambda x:random.randint(4200.00, 4400.00) if x==False else random.randint(170.00, 180.00))
    plan_result['deepOcpcBid'] = plan_result['optimizeDeepTrans'].apply(lambda x:2200 if x==True else np.nan)

    
    ad_info = []
    for i in range(plan_result.shape[0]):
        ad_info.append(json.loads(plan_result.loc[i, ['autoExpansion','education', 'customAge', 'keywords',
           'keywordsExtend', 'newInterests', 'sex', 'iosVersion', 'androidVersion',
           'excludeTrans', 'region', 'device', 'age']].to_json()))
    plan_result['audience'] = ad_info
    plan_result.drop(['autoExpansion','education', 'customAge', 'keywords',
           'keywordsExtend', 'newInterests', 'sex', 'iosVersion', 'androidVersion',
           'excludeTrans', 'region', 'device', 'age'], axis=1, inplace=True)
    ad_info = []
    for i in range(plan_result.shape[0]):
        ad_info.append(json.loads(plan_result.loc[i, [ 'transType', 'ocpcLevel',
           'isSkipStageOne', 'payMode', 'optimizeDeepTrans', 'transFrom',
           'ocpcBid', 'deepOcpcBid', 'deepTransType', 'roiRatio', 'useRoi']].to_json()))
    plan_result['ocpc'] = ad_info
    plan_result.drop([ 'transType', 'ocpcLevel',
           'isSkipStageOne', 'payMode', 'optimizeDeepTrans', 'transFrom',
           'ocpcBid', 'deepOcpcBid', 'deepTransType', 'roiRatio', 'useRoi'], axis=1, inplace=True)
    
#     plan_result['schedule'] = plan_result['ocpc'].apply(lambda x: [
#                                                                         {
#                                                                             "weekDay": 2,
#                                                                             "startHour": 5,
#                                                                             "endHour": 10
#                                                                         },
#                                                                         {
#                                                                             "weekDay": 3,
#                                                                             "startHour": 5,
#                                                                             "endHour": 10
#                                                                         },
#                                                                         {
#                                                                             "weekDay": 4,
#                                                                             "startHour": 5,
#                                                                             "endHour": 10
#                                                                         }
#                                                                     ])
    plan_result.to_csv('./plan_result.csv', index=0)  # 保存创建日志
    return plan_result

In [99]:
def get_ad_create(plan_result):
    ad_info = []
    for i in range(plan_result.shape[0]):
        ad_info.append(json.loads(plan_result.iloc[i].to_json()))
#     open_api_url_prefix = "https://ptom-pre.caohua.com/"
    open_api_url_prefix = "https://ptom.caohua.com/"
    uri = "model/generationPlanBatchTask"
    url = open_api_url_prefix + uri
    params = {
        "secretkey": "abc2018!@**@888",
        "mediaId": 45
    }
    rsp = requests.post(url, json=ad_info, params=params)
    rsp_data = rsp.json()
    print('结束....')
    return rsp_data

In [100]:
if __name__ == '__main__':
    ad_account_id_group = np.array([11633, 11634, 11635, 11636, 11637])   # 计划账号
    plan_num = 5   # 计划数量
    plan_result = get_plan(plan_num, ad_account_id_group)
    print('计划数量', plan_result.shape[0])
    get_ad_create(plan_result)

匹配的素材ID [45361 46078 46361 47487 47523]
计划数量 25
结束....
